In [ ]:
import camelot
import pandas as pd

In [ ]:
# PDF file to extract tables from
PU_file = "Opgoerelse-pcr-og-antigentest-24122020-b9x0.pdf"
#PrivateTestsFile = "foo.pdf"

In [ ]:
PU_tables = camelot.read_pdf(PU_file, pages = '2', flavor='stream')

In [ ]:
PU_data=PU_tables[0].df

In [ ]:
#remove wrong rows
PU_data=PU_data[5:-1]

In [ ]:
PU_data.columns = ['Date', 'Antal testudbydere', 'Antal PCR tests' , 'Antal positiv PCR tests','Positiv procent PCR' ,'Antal antigen tests','Antal positiv antigen tests','Positiv procent antigen']


In [ ]:
PU_data['Date']=pd.to_datetime(PU_data['Date'])
PU_data['Antal PCR tests'] = PU_data['Antal PCR tests'].str.replace('.', '').astype(float)
PU_data['Antal positiv PCR tests'] = PU_data['Antal positiv PCR tests'].str.replace('.', '').astype(float)
PU_data['Positiv procent PCR'] = PU_data['Positiv procent PCR'].str.replace(',', '.').str.replace('%', '').astype(float)
PU_data['Antal antigen tests'] = PU_data['Antal antigen tests'].str.replace('.', '').astype(float)
PU_data['Antal positiv antigen tests'] = PU_data['Antal positiv antigen tests'].str.replace('.', '').astype(float)
PU_data['Positiv procent antigen'] = PU_data['Positiv procent antigen'].str.replace(',', '.').str.replace('%', '').astype(float)
PU_data=PU_data.set_index(['Date'])

In [ ]:
PU_data

In [ ]:
PU_data.to_pickle('data_private.dat')

In [ ]:
PU_data.plot(y='Positiv procent antigen',style='.',color='green',label='From "Antigen Tests" (Quick tests)');